# preprocessing using pyspark

In [0]:
storage_account_name = 'usprojectprod'
storage_account_access_key = 'XeZ0aEiNiRwE4E9T0Vs9K6IoK+8eBSlwkLwauaa3jUAbvrat57FTUM/S6iakh4LvPF9Nt0k+Tb2i+ASt0nmk9Q=='

In [0]:
blob_container="finalproject"
storage_account_name="usprojectprod"
file_location = "wasbs://" + blob_container + "@" + storage_account_name + ".blob.core.windows.net/US_FINAL_PROJECT.csv"
file_type = "csv"

In [0]:
spark.conf.set(
  "fs.azure.account.key."+storage_account_name+".blob.core.windows.net",
  storage_account_access_key)

In [0]:
# reading data from blob container 
df = spark.read.csv(file_location, header=True, inferSchema=True)

In [0]:
# showing first 20 rows of dataframe
df.show(20)

+------------+----------+---------------+------------+--------------+--------+----------------------+--------------------+--------------------+----------+----------+--------------------+----------------+-----------------+---------------------+---------------------+-----------------+---------------------+---------------------+------------------+-------------------+-------------------+------------------+--------------------+--------------------+----------+-------------------+-------------------+-------------+-------------------+------------------+------------+-------------------+-------------------+---------+-------------------+-------------------+----------------+--------------------+--------------------+----------+--------------+--------------+------------------+--------------------+--------------------+------------------+-------------------+-------------------+-----------+---------------+---------------+-----------------------+---------------------------+---------------------------+--

In [0]:
# finding shape of data i.e. no of rows and columns

num_rows = df.count()
num_cols = len(df.columns)
print("Shape of DataFrame: ({}, {})".format(num_rows, num_cols))

Shape of DataFrame: (9002881, 58)


In [0]:
#finding count of null values present in each column of dataframe

from pyspark.sql.functions import col, sum as spark_sum
null_counts = df.select([spark_sum(col(c).isNull().cast('int')).alias(c) for c in df.columns])
null_counts.show()

+------------+----------+---------------+-----------+--------------+--------+----------------------+------+----+-----+----------+-------------+----------------+-----------------+---------------------+---------------------+-----------------+---------------------+---------------------+-----------+---------------+---------------+----------------+--------------------+--------------------+----------+--------------+--------------+-------------+-----------------+-----------------+------------+----------------+----------------+---------+-------------+-------------+----------------+--------------------+--------------------+----------+--------------+--------------+----------------+--------------------+--------------------+---------------+-------------------+-------------------+-----------+---------------+---------------+-----------------------+---------------------------+---------------------------+-------------------+------------------------------+------------+
|period_begin|period_end|period_d

In [0]:
# showing columns of dataframe
df.columns

['period_begin',
 'period_end',
 'period_duration',
 'region_type',
 'region_type_id',
 'table_id',
 'is_seasonally_adjusted',
 'region',
 'city',
 'state',
 'state_code',
 'property_type',
 'property_type_id',
 'median_sale_price',
 'median_sale_price_mom',
 'median_sale_price_yoy',
 'median_list_price',
 'median_list_price_mom',
 'median_list_price_yoy',
 'median_ppsf',
 'median_ppsf_mom',
 'median_ppsf_yoy',
 'median_list_ppsf',
 'median_list_ppsf_mom',
 'median_list_ppsf_yoy',
 'homes_sold',
 'homes_sold_mom',
 'homes_sold_yoy',
 'pending_sales',
 'pending_sales_mom',
 'pending_sales_yoy',
 'new_listings',
 'new_listings_mom',
 'new_listings_yoy',
 'inventory',
 'inventory_mom',
 'inventory_yoy',
 'months_of_supply',
 'months_of_supply_mom',
 'months_of_supply_yoy',
 'median_dom',
 'median_dom_mom',
 'median_dom_yoy',
 'avg_sale_to_list',
 'avg_sale_to_list_mom',
 'avg_sale_to_list_yoy',
 'sold_above_list',
 'sold_above_list_mom',
 'sold_above_list_yoy',
 'price_drops',
 'price_dro

In [0]:
# droping irrelevent columns which are not required for further analysis
columns_to_drop = ['period_duration', 'table_id', 'region_type', 'region_type_id', 'is_seasonally_adjusted', 'state_code', 'property_type_id', 'last_updated']
df = df.select([col for col in df.columns if col not in columns_to_drop])

In [0]:
# checking percentage of missing values present in each column of dataframe

from pyspark.sql.functions import col, sum
total_rows = df.count()
null_percentage_per_column = df.select([
    (sum(col(c).isNull().cast("int")) / total_rows * 100).alias(c + '_null_percentage')
    for c in df.columns
]).show()

+----------------------------+--------------------------+----------------------+--------------------+---------------------+-----------------------------+---------------------------------+-------------------------------------+-------------------------------------+---------------------------------+-------------------------------------+-------------------------------------+---------------------------+-------------------------------+-------------------------------+--------------------------------+------------------------------------+------------------------------------+--------------------------+------------------------------+------------------------------+-----------------------------+---------------------------------+---------------------------------+----------------------------+--------------------------------+--------------------------------+-------------------------+-----------------------------+-----------------------------+--------------------------------+---------------------------

In [0]:
# droping columns which have null values with more than 30 percent as filling null values will lead in data manupulation
from pyspark.sql.functions import col
total_rows = df.count()
threshold = int(total_rows * 0.7)
columns_to_keep = []
for column in df.columns:
    non_null_count = df.filter(col(column).isNotNull()).count()
    
    if non_null_count >= threshold:
        columns_to_keep.append(column)

df_filtered = df.select(columns_to_keep)
df_filtered.show()


+------------+----------+--------------------+--------------------+----------+--------------------+-----------------+---------------------+---------------------+-----------------+---------------------+---------------------+------------------+-------------------+-------------------+------------------+--------------------+--------------------+----------+-------------------+-------------------+------------+-------------------+-------------------+---------+-------------------+----------+--------------+--------------+------------------+--------------------+--------------------+------------------+-------------------+-------------------+-------------------+------------------------------+
|period_begin|period_end|              region|                city|     state|       property_type|median_sale_price|median_sale_price_mom|median_sale_price_yoy|median_list_price|median_list_price_mom|median_list_price_yoy|       median_ppsf|    median_ppsf_mom|    median_ppsf_yoy|  median_list_ppsf|median_li

In [0]:
# showing remaining columns
df_filtered.columns

['period_begin',
 'period_end',
 'region',
 'city',
 'state',
 'property_type',
 'median_sale_price',
 'median_sale_price_mom',
 'median_sale_price_yoy',
 'median_list_price',
 'median_list_price_mom',
 'median_list_price_yoy',
 'median_ppsf',
 'median_ppsf_mom',
 'median_ppsf_yoy',
 'median_list_ppsf',
 'median_list_ppsf_mom',
 'median_list_ppsf_yoy',
 'homes_sold',
 'homes_sold_mom',
 'homes_sold_yoy',
 'new_listings',
 'new_listings_mom',
 'new_listings_yoy',
 'inventory',
 'inventory_mom',
 'median_dom',
 'median_dom_mom',
 'median_dom_yoy',
 'avg_sale_to_list',
 'avg_sale_to_list_mom',
 'avg_sale_to_list_yoy',
 'sold_above_list',
 'sold_above_list_mom',
 'sold_above_list_yoy',
 'parent_metro_region',
 'parent_metro_region_metro_code']

In [0]:
# checking number of columns
len(df_filtered.columns)

37

In [0]:
# filling null values in column with median of that particular columns as all having null values are numerical columns
from pyspark.sql.functions import when, col, lit
import math
target_columns = ['median_sale_price',
 'median_sale_price_mom',
 'median_sale_price_yoy',
 'median_list_price',
 'median_list_price_mom',
 'median_list_price_yoy',
 'median_ppsf',
 'median_ppsf_mom',
 'median_ppsf_yoy',
 'median_list_ppsf',
 'median_list_ppsf_mom',
 'median_list_ppsf_yoy',
 'homes_sold',
 'homes_sold_mom',
 'homes_sold_yoy',
 'new_listings',
 'new_listings_mom',
 'new_listings_yoy',
 'inventory',
 'inventory_mom',
 'median_dom',
 'median_dom_mom',
 'median_dom_yoy',
 'avg_sale_to_list',
 'avg_sale_to_list_mom',
 'avg_sale_to_list_yoy',
 'sold_above_list',
 'sold_above_list_mom',
 'sold_above_list_yoy']
from pyspark.sql.functions import when, col

medians = df_filtered.approxQuantile(target_columns, [0.5], 0.25)[0]
median_dict = dict(zip(target_columns, medians))
for column in target_columns:
    if column not in df_filtered.columns:
        print(f"Column '{column}' not found in DataFrame")
        continue
    median = df_filtered.approxQuantile(column, [0.5], 0.25)[0]
    df_filtered = df_filtered.withColumn(column, when(col(column).isNull(), median).otherwise(col(column)))
df_filtered.show()


+------------+----------+--------------------+--------------------+----------+--------------------+-----------------+---------------------+---------------------+-----------------+---------------------+---------------------+------------------+-------------------+-------------------+------------------+--------------------+--------------------+----------+-------------------+-------------------+------------+-------------------+-------------------+---------+-------------------+----------+--------------+--------------+------------------+--------------------+--------------------+------------------+-------------------+-------------------+-------------------+------------------------------+
|period_begin|period_end|              region|                city|     state|       property_type|median_sale_price|median_sale_price_mom|median_sale_price_yoy|median_list_price|median_list_price_mom|median_list_price_yoy|       median_ppsf|    median_ppsf_mom|    median_ppsf_yoy|  median_list_ppsf|median_li

In [0]:
# checking percentage of null values in each column
from pyspark.sql.functions import col, sum
total_rows = df_filtered.count()
null_percentage_per_column = df_filtered.select([
    (sum(col(c).isNull().cast("int")) / total_rows * 100).alias(c + '_null_percentage')
    for c in df_filtered.columns
]).show()

+----------------------------+--------------------------+----------------------+--------------------+---------------------+-----------------------------+---------------------------------+-------------------------------------+-------------------------------------+---------------------------------+-------------------------------------+-------------------------------------+---------------------------+-------------------------------+-------------------------------+--------------------------------+------------------------------------+------------------------------------+--------------------------+------------------------------+------------------------------+----------------------------+--------------------------------+--------------------------------+-------------------------+-----------------------------+--------------------------+------------------------------+------------------------------+--------------------------------+------------------------------------+-----------------------------

In [0]:
# printing schema of all columns
df_filtered.printSchema()

root
 |-- period_begin: date (nullable = true)
 |-- period_end: date (nullable = true)
 |-- region: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- property_type: string (nullable = true)
 |-- median_sale_price: double (nullable = true)
 |-- median_sale_price_mom: double (nullable = true)
 |-- median_sale_price_yoy: double (nullable = true)
 |-- median_list_price: double (nullable = true)
 |-- median_list_price_mom: double (nullable = true)
 |-- median_list_price_yoy: double (nullable = true)
 |-- median_ppsf: double (nullable = true)
 |-- median_ppsf_mom: double (nullable = true)
 |-- median_ppsf_yoy: double (nullable = true)
 |-- median_list_ppsf: double (nullable = true)
 |-- median_list_ppsf_mom: double (nullable = true)
 |-- median_list_ppsf_yoy: double (nullable = true)
 |-- homes_sold: double (nullable = true)
 |-- homes_sold_mom: double (nullable = true)
 |-- homes_sold_yoy: double (nullable = true)
 |-- new_listings: doubl

In [0]:
# checking duplicate rows in dataframe
from pyspark.sql.functions import col
duplicates_count = df.groupBy(df.columns).count().filter(col('count') > 1).count()
print("Number of duplicate rows:", duplicates_count)

Number of duplicate rows: 1150


In [0]:
# dropping duplicate rows 
df = df.dropDuplicates()

In [0]:
from pyspark.sql.functions import col, sum

# Calculate the total number of rows in the DataFrame
total_rows = df_filtered.count()

# Calculate the number of null values per column and convert to percentage
null_percentage_per_column = df_filtered.select([
    (sum(col(c).isNull().cast("int")) / total_rows * 100).alias(c + '_null_percentage')
    for c in df_filtered.columns
]).show()

+----------------------------+--------------------------+----------------------+--------------------+---------------------+-----------------------------+---------------------------------+-------------------------------------+-------------------------------------+---------------------------------+-------------------------------------+-------------------------------------+---------------------------+-------------------------------+-------------------------------+--------------------------------+------------------------------------+------------------------------------+--------------------------+------------------------------+------------------------------+----------------------------+--------------------------------+--------------------------------+-------------------------+-----------------------------+--------------------------+------------------------------+------------------------------+--------------------------------+------------------------------------+-----------------------------

In [0]:
# rechecking as there are any null values and checking values
df_filtered.show()

+------------+----------+--------------------+--------------------+----------+--------------------+-----------------+---------------------+---------------------+-----------------+---------------------+---------------------+------------------+-------------------+-------------------+------------------+--------------------+--------------------+----------+-------------------+-------------------+------------+-------------------+-------------------+---------+-------------------+----------+--------------+--------------+------------------+--------------------+--------------------+------------------+-------------------+-------------------+-------------------+------------------------------+
|period_begin|period_end|              region|                city|     state|       property_type|median_sale_price|median_sale_price_mom|median_sale_price_yoy|median_list_price|median_list_price_mom|median_list_price_yoy|       median_ppsf|    median_ppsf_mom|    median_ppsf_yoy|  median_list_ppsf|median_li

In [0]:
# Storing processed data in blob container 
new_storage_account_name = "usprojectprod"
new_storage_account_access_key = "XeZ0aEiNiRwE4E9T0Vs9K6IoK+8eBSlwkLwauaa3jUAbvrat57FTUM/S6iakh4LvPF9Nt0k+Tb2i+ASt0nmk9Q=="
new_container_name = "processeddata"
new_file_path = "US_POWERBI.csv"

spark.conf.set(
    "fs.azure.account.key." + new_storage_account_name + ".blob.core.windows.net",
    new_storage_account_access_key
)
azure_blob_storage_path = "wasbs://processeddata@usprojectprod.blob.core.windows.net/"
df_filtered.repartition(1).write.mode("overwrite").option("header", "true").csv(azure_blob_storage_path)

# PYSQL

In [0]:
# creating temperory table
df_filtered.createOrReplaceTempView('US_REAL_ESTATE')



In [0]:
# query for max sale price and min sale price
result = spark.sql("SELECT MAX(median_sale_price) as maximum_sale_price, MIN(median_sale_price) as Minimum_sale_price FROM US_REAL_ESTATE")

result.show()

+------------------+------------------+
|maximum_sale_price|Minimum_sale_price|
+------------------+------------------+
|          2.3957E8|               1.0|
+------------------+------------------+



max_saleprice = 239,570,000

In [0]:
# querying avg sale price 
result = spark.sql("SELECT avg(median_sale_price) as average_sale_price FROM US_REAL_ESTATE")

result.show()

+------------------+
|average_sale_price|
+------------------+
|383798.26077124645|
+------------------+



In [0]:
# # querying avg sale price for each property type
result = spark.sql("SELECT avg(median_sale_price) as average_sale_price,property_type FROM US_REAL_ESTATE GROUP BY property_type")
result.show()

+------------------+--------------------+
|average_sale_price|       property_type|
+------------------+--------------------+
| 360700.5281633288|           Townhouse|
|417941.83691230125|Single Family Res...|
|303202.09799277794|         Condo/Co-op|
|418530.58684981195|Multi-Family (2-4...|
| 376556.2389648021|     All Residential|
+------------------+--------------------+



In [0]:
# # querying avg sale price for each year
result = spark.sql("SELECT avg(median_sale_price) as average_sale_price,year(period_end) FROM US_REAL_ESTATE GROUP BY year(period_end) ORDER BY year(period_end)")

result.show()

+------------------+----------------+
|average_sale_price|year(period_end)|
+------------------+----------------+
| 263449.7751705706|            2012|
| 289942.0454392559|            2013|
| 315106.6014558695|            2014|
|338544.78467745386|            2015|
| 361789.5536397878|            2016|
|388828.55332122813|            2017|
| 416210.4067015307|            2018|
| 424244.2878377079|            2019|
|448736.96008692513|            2020|
| 520889.7110136515|            2021|
+------------------+----------------+



In [0]:
# checking total number of sold properties from 2012 to 2021
result = spark.sql("SELECT count(median_sale_price) as no_of_sold_properties FROM US_REAL_ESTATE")

result.show()

+---------------------+
|no_of_sold_properties|
+---------------------+
|              9002881|
+---------------------+



In [0]:
# query for city with maximum sale price
result = spark.sql("SELECT city, max(median_sale_price) as maximum_sale_price FROM US_REAL_ESTATE GROUP BY city ORDER BY maximum_sale_price DESC LIMIT 1")
result.show()


+----------+------------------+
|      city|maximum_sale_price|
+----------+------------------+
|Ocean City|          2.3957E8|
+----------+------------------+



Ocean City : 239,570,000

In [0]:
# query for city with minimum sale price
result = spark.sql("SELECT city, min(median_sale_price) as minimum_sale_price FROM US_REAL_ESTATE GROUP BY city ORDER BY minimum_sale_price DESC LIMIT 1")
result.show()

+-----------+------------------+
|       city|minimum_sale_price|
+-----------+------------------+
|Toro Canyon|         2532000.0|
+-----------+------------------+



In [0]:
# query for maximum listed price and minimum listed price
from pyspark.sql import functions as F
result = spark.sql("SELECT MAX(median_list_price) as maximum_listed_price, MIN(median_list_price) as Minimum_listed_price FROM US_REAL_ESTATE")
result.show()

+--------------------+--------------------+
|maximum_listed_price|Minimum_listed_price|
+--------------------+--------------------+
|        9.99999999E8|               100.0|
+--------------------+--------------------+



999999999.0

In [0]:
# query for average listed price
result = spark.sql("SELECT avg(median_list_price) as average_listed_price  FROM US_REAL_ESTATE")
result.show()

+--------------------+
|average_listed_price|
+--------------------+
|  393705.01265894773|
+--------------------+



In [0]:
# query for average listed price for each property type
result = spark.sql("SELECT avg(median_list_price) as average_listed_price,property_type  FROM US_REAL_ESTATE group by property_type")
result.show()

+--------------------+--------------------+
|average_listed_price|       property_type|
+--------------------+--------------------+
|  355592.48143801483|           Townhouse|
|  430429.11517231335|Single Family Res...|
|  313655.82373490994|         Condo/Co-op|
|  410895.24590747594|Multi-Family (2-4...|
|  391066.17915906373|     All Residential|
+--------------------+--------------------+



In [0]:
# # query for average listed price for each year
result = spark.sql("SELECT avg(median_list_price) as average_listed_price,year(period_end)  FROM US_REAL_ESTATE group by year(period_end) ORDER BY year(period_end)")
result.show()

+--------------------+----------------+
|average_listed_price|year(period_end)|
+--------------------+----------------+
|  288723.67849422275|            2012|
|  310458.02316994994|            2013|
|   329743.3819421097|            2014|
|  351884.52596548694|            2015|
|  374823.56203456025|            2016|
|  399892.09859415697|            2017|
|  420825.30367595877|            2018|
|  428943.96574527776|            2019|
|     456677.29090461|            2020|
|   513035.4793163661|            2021|
+--------------------+----------------+



In [0]:
# top cities with number of new listings
result = spark.sql("SELECT count(new_listings) as No_listings,state  FROM US_REAL_ESTATE group by state ORDER BY No_listings DESC")
result.show()

+-----------+--------------+
|No_listings|         state|
+-----------+--------------+
|    1484367|    California|
|    1076668|       Florida|
|     634310|         Texas|
|     384859|      Illinois|
|     328935|       Arizona|
|     302380|      Colorado|
|     295443| Massachusetts|
|     267679|      New York|
|     266339|          Ohio|
|     262468|    Washington|
|     257570|North Carolina|
|     222533|  Pennsylvania|
|     220306|      Virginia|
|     217342|      Michigan|
|     207887|       Georgia|
|     184145|     Minnesota|
|     174989|     Wisconsin|
|     160696|        Oregon|
|     154264|     Tennessee|
|     148048|     Louisiana|
+-----------+--------------+
only showing top 20 rows



In [0]:
# number of listings for each property type
result = spark.sql("SELECT count(new_listings) as No_listings,property_type  FROM US_REAL_ESTATE group by property_type ORDER BY No_listings DESC")
result.show()

+-----------+--------------------+
|No_listings|       property_type|
+-----------+--------------------+
|    3372051|     All Residential|
|    3082872|Single Family Res...|
|    1058976|         Condo/Co-op|
|     816292|           Townhouse|
|     672690|Multi-Family (2-4...|
+-----------+--------------------+



In [0]:
# query for total inventory in each year
result = spark.sql("SELECT count(inventory) as total_inventory,year(period_end) as YEAR FROM US_REAL_ESTATE group by year(period_end) order by  YEAR")
result.show()

+---------------+----+
|total_inventory|YEAR|
+---------------+----+
|         651783|2012|
|         820524|2013|
|         841765|2014|
|         911017|2015|
|         941717|2016|
|         953307|2017|
|         958199|2018|
|         956744|2019|
|         957606|2020|
|        1010219|2021|
+---------------+----+



In [0]:
# query fortop 10 cities having average sales price 
result = spark.sql("SELECT city,avg(median_sale_price) as avg_sale_price  FROM US_REAL_ESTATE group by city order by avg_sale_price DESC LIMIT 10")
result.show()


+-----------------+--------------------+
|             city|      avg_sale_price|
+-----------------+--------------------+
|      Toro Canyon|1.0742489328877006E7|
|         Atherton|   5506627.661482159|
|           Makena|   5147386.970679012|
|      Gulf Stream|   5091264.050420168|
|     Hillsborough|   4391806.822213653|
|    Lake Sherwood|   3799474.358974359|
|  Los Altos Hills|           3794157.5|
|Belvedere Tiburon|  3743529.6614583335|
|   Portola Valley|  3477829.1381418095|
|        Montecito|  3407654.8757894738|
+-----------------+--------------------+



In [0]:
# year with the highest total homes_sold
result = spark.sql("SELECT YEAR(period_end) as year, SUM(homes_sold) as total_homes_sold FROM US_REAL_ESTATE GROUP BY year ORDER by total_homes_sold DESC LIMIT 1")
result.show()

+----+----------------+
|year|total_homes_sold|
+----+----------------+
|2021|     1.7466442E7|
+----+----------------+



In [0]:
#property_type with the highest average inventory
result = spark.sql("SELECT property_type, AVG(inventory) as avg_inventory FROM US_REAL_ESTATE GROUP BY property_type ORDER BY avg_inventory DESC LIMIT 1")
result.show()

+---------------+------------------+
|  property_type|     avg_inventory|
+---------------+------------------+
|All Residential|21.578660287166475|
+---------------+------------------+



In [0]:
# state with the most new_listings in the latest year
result=spark.sql("SELECT state, SUM(new_listings) as total_new_listings FROM US_REAL_ESTATE WHERE YEAR(period_end) = (SELECT MAX(YEAR(period_end)) FROM US_REAL_ESTATE) GROUP BY state ORDER BY total_new_listings DESC LIMIT 1")
result.show()

+----------+------------------+
|     state|total_new_listings|
+----------+------------------+
|California|         2636011.0|
+----------+------------------+



In [0]:
#average median_sale_price change month over month (median_sale_price_mom) and year over year (median_sale_price_yoy)
result=spark.sql("SELECT AVG(median_sale_price_mom) as avg_mom_change, AVG(median_sale_price_yoy) as avg_yoy_change FROM US_REAL_ESTATE")
result.show()

+-------------------+------------------+
|     avg_mom_change|    avg_yoy_change|
+-------------------+------------------+
|0.14580845037048032|0.9312429727288867|
+-------------------+------------------+



In [0]:
#city and year with the highest sold_above_list
result=spark.sql("SELECT city, YEAR(period_end) as year, MAX(sold_above_list) as max_sold_above_list FROM US_REAL_ESTATE GROUP BY city, year ORDER BY max_sold_above_list DESC LIMIT 1")
result.show()


+---------+----+-------------------+
|     city|year|max_sold_above_list|
+---------+----+-------------------+
|Lancaster|2012|                1.0|
+---------+----+-------------------+



In [0]:
#city and year with the highest sale_price value
result=spark.sql("SELECT city, YEAR(period_end) as year, MAX(median_sale_price) as max_price_sold FROM US_REAL_ESTATE GROUP BY city, year ORDER BY max_price_sold DESC LIMIT 1")
result.show()
# max_price : 239,570,000

+----------+----+--------------+
|      city|year|max_price_sold|
+----------+----+--------------+
|Ocean City|2017|      2.3957E8|
+----------+----+--------------+



In [0]:
#total number of homes_sold for each parent_metro_region
result=spark.sql("SELECT parent_metro_region, SUM(homes_sold) as total_homes_sold FROM US_REAL_ESTATE GROUP BY parent_metro_region ORDER BY total_homes_sold DESC limit 10")
result.show()

+-------------------+----------------+
|parent_metro_region|total_homes_sold|
+-------------------+----------------+
|        Phoenix, AZ|       6517274.0|
|        Chicago, IL|       6387050.0|
|    Los Angeles, CA|       5980740.0|
|        Houston, TX|       4329168.0|
|         Denver, CO|       4024012.0|
|       New York, NY|       3498858.0|
|    San Antonio, TX|       3212670.0|
|        Seattle, WA|       2820576.0|
|      San Diego, CA|       2816976.0|
|     Washington, DC|       2733198.0|
+-------------------+----------------+



In [0]:
#the property_type with the lowest average median_dom (Days on Market)
result=spark.sql("SELECT property_type, AVG(median_dom) as avg_days_on_market FROM US_REAL_ESTATE GROUP BY property_type ORDER BY avg_days_on_market ASC LIMIT 1")
result.show()


+-------------+------------------+
|property_type|avg_days_on_market|
+-------------+------------------+
|    Townhouse| 73.51006196312105|
+-------------+------------------+

